In [1]:
from datetime import datetime
from os.path import exists as file_exists
import pandas as pd
from pprint import pprint as pp
import re
import requests
from time import sleep
import xmltodict
import Credentials      # Get API keys, etc.

apikey = Credentials.prod_api
baseurl = 'https://api-na.hosted.exlibrisgroup.com'
item_query = '/almaws/v1/bibs/{mms_id}/holdings/{holding_id}/items/{item_pid}?apikey={apikey}'

exported_csv = "FullItemList.csv"
filled_csv = "FilledEnumChron.csv"
err_log_txt = "log.txt"

### Run this cell if you want to use the Limited-list CSV
exported_csv = "LimitedList.csv"

#### Create original df

In [133]:
# Depending on how the file was exported, column names may or may have either spaces or underscores
df = pd.read_csv(exported_csv, dtype=str)

#### Clean up, tweak & format df

In [134]:
# Remove spaces from column names
df.columns = [c.replace(' ', '_') for c in df.columns]
# Rename certain columns
df = df.rename(columns={'Permanent_Location': 'Location', 'Item_Policy': 'Policy', 'Material_Type': 'Material'})
# Strip leading/trailing space from Description
df.Description = df.Description.str.strip()
# Collapse multiple spaces within the Description
df.Description.replace(' +', ' ', regex=True, inplace=True)

In [135]:
# Add columns for the Enum/Chron fields
EC_fields = ['Enum_A', 'Enum_B', 'Chron_I', 'Chron_J']
df[EC_fields] = None

#### The function for getting info from Description to Enum/Chron fields:

In [136]:
def fill_and_extract(regex, these_fields):
    exp = re.compile(regex, re.IGNORECASE)
    for i, f in enumerate(these_fields):
        df[f] = df['Description'].str.extract(exp, expand=True)[i].fillna(df[f])

#### Set some common expressions that can be used in a modular way

In [137]:
# Month/season
mmmRE = r'(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|June?|July?|Aug(?:ust)?|Sept?(?:ember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?|Spr(?:ing)?|Sum(?:mer)?|Fall?|Aut(?:umn)?|Win(?:ter)?)'
# Month + date(s)
mmm_ddRE = mmmRE + r'(?: \d{1,2}(?:[\-\/]\d{1,2})?)?'
# 4-digit year/range of years (post-18th-century)
yyyyRE = r'(?:1[89]|20)\d{2}'
# 4-digit year leading to a range
yyyy_yyRE = r'(?:1[89]|20)\d{2}-(?:\d{2}|\d{4})'

#### Call it repeatedly for each regex that you come up with:
**[Test your Regex here](https://regex101.com/)**

In [123]:
# Just volume(s) & nothing else
fill_and_extract(r'^v\. ?(\d+[a-z]?(?:[\&\-]\d+)?)$',
                 ['Enum_A'])

In [138]:
# Volume + issue
fill_and_extract(r'^v\. ?(\d+)[ \/]no\. ?(\d+)$',
                 ['Enum_A', 'Enum_B'])

In [129]:
# Vol + issue + date + year
fill_and_extract(r'^v\. ?(\d+)[ \/]no\. ?(\d+) (' + mmm_ddRE + r'),? (' + yyyyRE + ')$',
                 ['Enum_A', 'Enum_B', 'Chron_J', 'Chron_I'])

In [17]:
# Volume + year(s)
fill_and_extract(r'^v\. ?(\d+) +(\(?' + yyyy_yyRE + r'\)?)$',
                 ['Enum_A', 'Chron_I'])

In [127]:
# Just issue & nothing else
fill_and_extract(r'^no\. ?(\d+)$',
                 ['Enum_B'])

In [26]:
# Issue + year(s)
fill_and_extract(r'^no\. ?(\d+) (' + yyyy_yyRE + r')$',
                 ['Enum_B', 'Chron_I'])

In [102]:
# Range of dates within one calendar year
fill_and_extract(r'^(' + mmm_ddRE + r'[\-\/]' + mmm_ddRE + '), (' + yyyyRE + ')$',
                 ['Chron_J', 'Chron_I'])

In [95]:
# Date + year
fill_and_extract(r'^(' + mmm_ddRE + '),? (' + yyyyRE + ')$',
                 ['Chron_J', 'Chron_I'])

In [8]:
# Just year/range of years
fill_and_extract(r'^(' + yyyy_yyRE + r')$',
                 ['Chron_I'])

In [44]:
# Year(s) + volume(s)
fill_and_extract(r'^(' + yyyy_yyRE + r') v\. ?(\d+(?:[\-\/]\d+)?)$',
                 ['Chron_I', 'Enum_A'])

In [111]:
# Year + month/season/date
fill_and_extract(r'^(' + yyyyRE + r') (' + mmm_ddRE + r')$',
                 ['Chron_I', 'Chron_J'])

#### Then once all those replacements are done, pull filled-in records out to a new dataframe

In [139]:
# Create a dataframe to hold JUST records that get filled
filled = pd.DataFrame()
# Populate the new dataframe with any records that now have at least one Enum/Chron field filled
filled = df.dropna(subset=EC_fields, thresh=1)

#### Then apply the changes via the API and log filled items to the Filled CSV

In [140]:
records = len(filled)
c = 0
needs_header=not file_exists(filled_csv) # Apparently we're creating the file, so it needs a header

In [141]:
### Run this bit to see what got "filled" before hitting the API

for index, row in filled.fillna('').iterrows():
    c += 1
    print(c, ' / '.join([row['MMS_ID'], row['Holdings_ID'], row['Item_ID']]),
          str(row['Description']),
          ' | '.join(x or '' for x in [row['Enum_A'], row['Enum_B'], row['Chron_I'], row['Chron_J']]),
          sep="\t")


In [ ]:
with open(err_log_txt, 'a') as err_log:
    for index, row in filled.fillna('').iterrows():
        c += 1
        r = requests.get(''.join([baseurl,
                                  item_query.format(mms_id=str(row['MMS_ID']),
                                                    holding_id=str(row['Holdings_ID']),
                                                    item_pid=str(row['Item_ID']),
                                                    apikey=apikey)]))
        rdict = xmltodict.parse(r.text)
        if r.status_code == 429:  # Too many requests--daily limit
            print()
            print('Reached API request limit for today. Stopping execution.')
            print()            
            ## Drop this record & everything after from "filled"
            filled = filled.iloc[:c-1]
            break
        if r.status_code != 200:
            e = xmltodict.parse(r._content)
            # Log the error
            print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                  ' Error FETCHING item ', row['Item_ID'], ': (', r.status_code, ') ',
                  e['web_service_result']['errorList']['error']['errorMessage'],
                  sep='',
                  file=err_log)
            # Remove this item from the "filled" df
            filled = filled.drop([index])
            continue
        if (c % (records/100) < 1):
            print(int(100*c/records), '% complete', sep='')#, end='\r')
            sleep(5)
            
        # Merge derived values into the retrieved data (rdict)
        rdict['item']['item_data']['enumeration_a'] = str(row['Enum_A'])
        rdict['item']['item_data']['enumeration_b'] = str(row['Enum_B'])
        rdict['item']['item_data']['chronology_i'] = str(row['Chron_I'])
        rdict['item']['item_data']['chronology_j'] = str(row['Chron_J'])
        # Set an internal note, if there's an empty one available
        if ('Enum/Chron derived from Description' not in rdict['item']['item_data'].values()):
            if (not rdict['item']['item_data']['internal_note_1']):
                rdict['item']['item_data']['internal_note_1'] = 'Enum/Chron derived from Description'
            elif (not rdict['item']['item_data']['internal_note_2']):
                rdict['item']['item_data']['internal_note_2'] = 'Enum/Chron derived from Description'
            elif (not rdict['item']['item_data']['internal_note_3']):
                rdict['item']['item_data']['internal_note_3'] = 'Enum/Chron derived from Description'
            else: # Nbd, just log it
                print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                      ' No internal note available for item MMS ID ',
                      str(row['MMS_ID']), sep="", file=err_log)
 
        # Push the altered record back into Alma
        pxml = xmltodict.unparse(rdict)
        p = requests.put(''.join([baseurl,
                                  item_query.format(mms_id=row['MMS_ID'],
                                                    holding_id=row['Holdings_ID'],
                                                    item_pid=row['Item_ID'],
                                                    apikey=apikey)]),
                         data=pxml.encode('utf-8'), headers={'Content-Type': 'application/xml'})
        if r.status_code == 429:  # Too many requests--daily limit
            print()
            print('Reached API request limit for today. Stopping execution.')
            print()            
            ## Drop this record & everything after from "filled"
            filled = filled.iloc[:c-1]
            break
        if p.status_code != 200:
            e = xmltodict.parse(p._content)
            # Log the error
            print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"), ' Error UPDATING item ', row['Item_ID'], ': (', p.status_code, ') ',
                e['web_service_result']['errorList']['error']['errorMessage'],
                sep='', file=err_log)
            # Remove this item from the "filled" df
            filled = filled.drop([index])
            continue
        print(c, ' / '.join([row['MMS_ID'], row['Holdings_ID'], row['Item_ID']]),
              row['Description'],
              ' | '.join(x or '' for x in [row['Enum_A'], row['Enum_B'], row['Chron_I'], row['Chron_J']]),
              sep="\t")

        # Log it to the CSV
        #    Btw, the 'to_frame().T' transposes it, so it all goes in as a single comma-separated row
        row.to_frame().T.to_csv(filled_csv, mode='a', index=False, header=needs_header)
        needs_header = False # Henceforth

#### Purge filled rows from the original CSV

In [53]:
# Purge filled records from the original df
df = df.loc[~df['Item_ID'].isin(filled['Item_ID'])]

# Re-create the original CSV from that df
df.to_csv(exported_csv, index=False) # By default, will overwrite

# View df & filled

In [ ]:
display(df)
display(filled)

# <span style="color:#cc0000">Undo</span> changes to some records!

In [ ]:
undo_csv="Undo.csv"
df = pd.read_csv(undo_csv, converters={'Item_ID': str, 'Holdings_ID': str, 'MMS_ID': str, 'Item ID': str, 'Holdings ID': str, 'MMS ID': str, 'Enum_A': str, 'Enum_B': str, 'Chron_I': str, 'Chron_J': str})
# Remove spaces from column names
df.columns = [c.replace(' ', '_') for c in df.columns]
# Rename certain columns
df = df.rename(columns={'Permanent_Location': 'Location', 'Item_Policy': 'Policy', 'Material_Type': 'Material'})
records = len(df)
c = 0

for index, row in df.fillna('').iterrows():
    c += 1
    
    # Get the current record
    print(c, ' / '.join((row['MMS_ID'], row['Holdings_ID'], row['Item_ID'])), str(row['Description']), sep="\t")
    r = requests.get(''.join([baseurl,
                              item_query.format(mms_id=str(row['MMS_ID']),
                                                holding_id=str(row['Holdings_ID']),
                                                item_pid=str(row['Item_ID']),
                                                apikey=apikey)]))
    rdict = xmltodict.parse(r.text)
    if r.status_code != 200:
        e = xmltodict.parse(r._content)
        # Output the error
        print('Error FETCHING item ', row['Item_ID'], ': (', r.status_code, ') ',
            e['web_service_result']['errorList']['error']['errorMessage'],
            sep='')
        continue
    if (c % (records/100) < 1):
        print(int(100*c/records), '% complete', sep='')#, end='\r')
        sleep(5)

    # Merge derived values into the retrieved data (rdict)
    rdict['item']['item_data']['enumeration_a'] = \
        rdict['item']['item_data']['enumeration_b'] = \
        rdict['item']['item_data']['chronology_i'] = \
        rdict['item']['item_data']['chronology_j'] = None
    # Set an internal note, if there's an empty one available
    if (rdict['item']['item_data']['internal_note_1'] == 'Enum/Chron derived from Description'):
        rdict['item']['item_data']['internal_note_1'] = None
    if (rdict['item']['item_data']['internal_note_2'] == 'Enum/Chron derived from Description'):
        rdict['item']['item_data']['internal_note_2'] = None
    if (rdict['item']['item_data']['internal_note_3'] == 'Enum/Chron derived from Description'):
        rdict['item']['item_data']['internal_note_3'] = None

    # Push the altered record back into Alma
    pxml = xmltodict.unparse(rdict)
    p = requests.put(''.join([baseurl,
                              item_query.format(mms_id=row['MMS_ID'],
                                                holding_id=row['Holdings_ID'],
                                                item_pid=row['Item_ID'],
                                                apikey=apikey)]),
                     data=pxml.encode('utf-8'), headers={'Content-Type': 'application/xml'})
    if p.status_code != 200:
        e = xmltodict.parse(p._content)
        # Log the error
        print('Error UPDATING item ', row['Item_ID'], ': (', p.status_code, ') ',
            e['web_service_result']['errorList']['error']['errorMessage'],
            sep='')
        continue


# Testing stuff

In [45]:
desc = 'Spring 1885'
mmmRE = r'(Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|June?|July?|Aug(?:ust)?|Sept?(?:ember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?|Spr(?:ing)?|Sum(?:mer)?|Fall?|Aut(?:umn)?|Win(?:ter)?)'
yyyyRE = r'(?:1[89]|20)\d\d(?:[\-\/](?:1[89]|20)?\d\d)?'
exp = re.compile('^' + mmmRE + ' *' + yyyyRE + '$')

exp.match(desc)

<re.Match object; span=(0, 11), match='Spring 1885'>

In [54]:
fne = r'^v\. ?(\d+)[ \/]no\. ?(\d+) (' + mmmRE + r' \d{1,2}),? '+'(' + yyyyRE + ')$', ['Enum_A', 'Enum_B', 'Chron_J', 'Chron_I'])

In [55]:
df

,Location,Material,Policy,Title,MMS_ID,Holdings_ID,Item_ID,Description,Enum_A,Enum_B,Chron_I,Chron_J
0,1L Cataloging,Other,Serial,Cataloging service bulletin index,991001247819706388,2267069970006388,2367069950006388,no.1-128 1978-2010,None,None,None,None
1,1L Cataloging,Other,Serial,The Music OCLC Users Group presents the best o...,991000134449706388,2276843580006388,2376843570006388,2000,None,None,None,None
2,1L Magazine Stand,Issue,Non Circulating,The antelope newspaper,991001196859706388,22130215730006388,23130215550006388,v.123 no.16 Mar 23 2022,123,16,Mar,Mar 23
3,1L Magazine Stand,Issue,Non Circulating,The antelope newspaper,991001196859706388,22130215730006388,23130215560006388,v.123 no.14 Mar 2 2022,123,14,Mar,Mar 2
4,1L Magazine Stand,Issue,Non Circulating,The antelope newspaper,991001196859706388,22130215730006388,23130215570006388,v.123 no.15 Mar 9 2022,123,15,Mar,Mar 9
...,...,...,...,...,...,...,...,...,...,...,...,...
104235,Special Collection,Other,Serial,Work paper,991000171229706388,2266649730006388,2366649680006388,no.20 Spr 1976,None,None,None,None
104236,Special Collection,Other,Serial,Work paper,991000171229706388,2266649730006388,2366649690006388,no.19 Dec 1975,None,None,None,None
104237,Special Collection,Other,Serial,Work paper,991000171229706388,2266649730006388,2366649700006388,no.18 Sep 1975,None,None,None,None
104238,Special Collection,Other,Serial,Work paper,991000171229706388,2266649730006388,2366649710006388,no.13 Dec 1973,None,None,None,None
